In [2]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# Mining


## Problem Description

A mining company needs to create a five-year operating plan for a certain area with four mines in it.

They can only operate a maximum of three mines in this area in any given year. However, even though a mine may not operate in a given year, the company still must pay royalties on that mine if there is an expectation that it will operate again in the future. Otherwise, it can be permanently closed and no more royalties need to be paid.

The yearly royalties due for each open mine (operating or not) are as follows:

| <i></i> | Royalties |
| --- | --- |
| Mine 1 | $\$5 Million$ |
| Mine 2 | $\$4 Million$ |
| Mine 3 | $\$4 Million$ |
| Mine 4 | $\$5 Million$ |

There is a maximum amount of ore that can be extracted from each mine in a given year. These limits are as follows:

| <i></i> | Max Production |
| --- | --- |
| Mine 1 | $2.0\times10^6$ Tons |
| Mine 2 | $2.5\times10^6$ Tons |
| Mine 3 | $1.3\times10^6$ Tons |
| Mine 4 | $3.0\times10^6$ Tons |

Each mine produces a different grade of ore. This grade is measured on a scale such that blending ores together results in a linear combination of the quality requirements. For example, if equal quantities of ore from two different mines were combined, the resulting ore would have a grade that is the average of the grade for each of the two ores. The grade for each mine’s ore is as follows:

| <i></i> | Ore Quality |
| --- | --- |
| Mine 1 | 1.0 |
| Mine 2 | 0.7 |
| Mine 3 | 1.5 |
| Mine 4 | 0.5 |


Each year, the ore produced from each operating mine must be combined to produce ore of a certain grade. The yearly objectives for the combined ore are as follows:

| <i></i> | Quality Target |
| --- | --- |
| Year 1 | 0.9 |
| Year 2 | 0.8 |
| Year 3 | 1.2 |
| Year 4 | 0.6 |
| Year 5 | 1.0 |

The final blended ore sells for $\$10$/ton. Revenues and costs for future years are discounted at the rate of $10\%$ per annum.

The key question for the mining company is: Which mines should be operated each year and how much ore should be extracted from each mine?

This problem is based on a larger one faced by the firm of English China Clays, which had to in decide which mines to work. In that problem (in the 1970s), the goal was to work up to four mines out of 20 in each year.

## Model Formulation

### Sets and Indices

$t \in \text{Years}=\{1,2,\dots,5\}$: Set of years.

$m \in \text{Mines}=\{1,2,\dots,4\}$: Set of mines.

### Parameters

$\text{price} \in \mathbb{R}^+$: Selling price (in USD) of one ton of blended ore.

$\text{max_mines} \in \mathbb{N}$: Maximum number of mines that can operate in any given year.

$\text{royalties}_m \in \mathbb{R}^+$: Yearly royalties (in USD) for having mine $m$ open.

$\text{capacity}_m \in \mathbb{R}^+$: Maximum tons of ore that can be extracted from mine $m$ in any given year.

$\text{quality}_m \in \mathbb{R}^+$: Quality of the ore extracted from mine $m$.

$\text{target} \in \mathbb{R}^+$: Quality target of the blended ore in year $t$.

$\text{time_discount}_t \in [0,1] \subset \mathbb{R}^+$: Time discount for revenue and cost in year $t$.

### Decision Variables

$\text{blend}_t \in \mathbb{R}^+$: Tons of blended ore in year $t$.

$\text{extract}_{t,m} \in \mathbb{R}^+$: Tons of ore extracted from mine $m$ in year $t$.

$\text{working}_{t,m} \in \{0,1\}$: 1 if mine $m$ is working in year $t$, 0 otherwise.

$\text{available}_{t,m} \in \{0,1\}$: 1 if mine $m$ is open in year $t$, 0 otherwise.



### 1. If mining company only operates in the first year, how to maximize the total profit?

### Objective Function

**Profits**: Maximize profits from the extraction of ore.

- **Profit**: Maximize the total profit (in USD) of the first year.

\begin{equation}
\text{Maximize} \quad Z = \sum_{m \in \text{Mines}}{\text{extract}_m*\text{price}-\sum_{m \in \text{Mines}}{\text{royalties}_m*\text{working}_m}}
\tag{0}
\end{equation}

### Constraints

- **Operating Mines**: The total number of operating mines cannot exceed the limit.

\begin{equation}
\sum_{m \in \text{Mines}}{\text{working}_m} \leq \text{max_mines} \quad 
\tag{1}
\end{equation}

- **Quality**: The final quality of the ore blended in year $t$ must meet the target.

\begin{equation}
\sum_{m \in \text{Mines}}{\text{quality}_m*\text{extract}_m} =\sum_{m \in \text{Mines}}\text{target}*\text{extract}_m 
\tag{2}
\end{equation}

- **Mine Capacity**: Total tons of ore extracted from mine $m$ in year $t$ cannot exceed the yearly capacity of that mine.

\begin{equation}
\sum_{m \in \text{Mines}}{\text{extract}_m} \leq \text{capacity}_m*\text{working}_m 
\tag{3}
\end{equation}

In [3]:
mine, quality, royalties, max_production = gp.multidict({
    1: [1.0, 5000000, 2000000],
    2: [0.7, 4000000, 2500000],
    3: [1.5, 4000000, 1300000], 
    4: [0.5, 5000000, 3000000]
})
max_mines = 3
price = 10
quality_target= 0.9

In [4]:
model1 = gp.Model('Max_first_year')

extract = model1.addVars(mine, name="Extract")
working = model1.addVars(mine, vtype=GRB.BINARY, name="Working")


Restricted license - for non-production use only - expires 2023-10-25


In [5]:
 model1.setObjective((gp.quicksum(10 * extract[i] for i in mine)\
                      - gp.quicksum(royalties[i]*-working[i] for i in mine)), 
                   GRB.MAXIMIZE)
    


In [6]:
# can only operate a maximum of three mines in this area in any given year

max_mineConstr = model1.addConstr(gp.quicksum(working[i] for i in mine)<=3, name='max_mineConstr')

In [7]:
# quality of the blended ore needs to be 0.9

qualityConstr = model1.addConstr((gp.quicksum(extract[i]*quality[i] for i in mine) \
                                 == quality_target * gp.quicksum(extract[i] for i in mine)), \
name='qualityConstr')


In [8]:
# the mine can extract no more than the extract limit and also 
# that there is only an output if the mine is operational

limitConstr = model1.addConstrs((extract[i] <= working[i]*max_production[i] for i in mine), name='limitConstr')

In [9]:
# Run optimization engine

model1.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6 rows, 8 columns and 16 nonzeros
Model fingerprint: 0xdcc57c51
Variable types: 4 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e-01, 3e+06]
  Objective range  [1e+01, 5e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+00]
Found heuristic solution: objective 1.300000e+07
Presolve time: 0.00s
Presolved: 6 rows, 8 columns, 16 nonzeros
Variable types: 4 continuous, 4 integer (4 binary)
Found heuristic solution: objective 1.400000e+07

Root relaxation: objective 7.440909e+07, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 7.4409e+07    0    2 1.4000e+07 7.4409e+07   431%     -    0s
H    0     0                    6.300000e+07 7

In [10]:
for x in model1.getVars():
    print(x.varName, x.x)

Extract[1] 2000000.0
Extract[2] 1.862645149230957e-09
Extract[3] 1300000.0
Extract[4] 2449999.999999999
Working[1] 1.0
Working[2] 0.0
Working[3] 1.0
Working[4] 1.0


In [11]:
print('Obj: ' + str(model1.objVal))

Obj: 71500000.00000001


### 2. If mining company only operates in the first year, and decides to sell the raw ore without mixing. Which mines they should operate to maximize the total profit? The market price for raw ore are as follow:

| <i></i> | Raw Ore price |
| --- | --- |
| Mine 1 | 10 |
| Mine 2 | 9 |
| Mine 3 | 11 |
| Mine 4 | 8 |

### Objective function

\begin{equation}
\text{Maximize} \quad Profit = \sum_{m \in \text{Mines}}{(\text{price}_m*\text{extract}_m-\text{royalties}_m*\text{working}_{m})}
\tag{0}
\end{equation}

### Constraints
- **Operating Mines**: The total number of operating mines cannot exceed the limit.

\begin{equation}
\sum_{m \in \text{Mines}}{\text{working}_{m}} \leq \text{max_mines}
\tag{1}
\end{equation}

- **Mine Capacity**: Total tons of ore extracted from mine $m$ cannot exceed the yearly capacity of that mine.

\begin{equation}
\sum_{m \in \text{Mines}}{\text{extract}_{m}} \leq \text{capacity}_m*\text{working}_{m}
\tag{2}
\end{equation}

- **Total profit**: Total Profit cannot be less than 64M dollars.

\begin{equation}
\sum_{m \in \text{Mines}}{(\text{price}_m*\text{extract}_m-\text{royalties}_m*\text{working}_{m})}>=64,000,000
\tag{3}
\end{equation}



In [12]:
year, goal = gp.multidict({
    1: 0.9,
    2: 0.8,
    3: 1.2,
    4: 0.6,
    5: 1.0
})

mine, quality, royalties, maxproduction,price = gp.multidict({
    1: [1.0, 5000000, 2000000,10],
    2: [0.7, 4000000, 2500000,9],
    3: [1.5, 4000000, 1300000,11], 
    4: [0.5, 5000000, 3000000,8]
})

In [13]:
# Parameters

max_mines = 3

In [14]:
model2= gp.Model('most_profit_and_grade')

In [15]:
blend = model2.addVars(year, name="Blend")
extract = model2.addVars(mine, name="Extract")
working = model2.addVars(mine, vtype=GRB.BINARY, name="Working")
available = model2.addVars(mine, vtype=GRB.BINARY, name="Available")

In [16]:
# Operating Mines

OperatingMines  = model2.addConstr((gp.quicksum(working[m] for m in mine ) <= max_mines), "Operating_mines")

# Mass Conservation

MassConservation = model2.addConstr((gp.quicksum(extract[m]for m in mine ) == blend[1]), "Mass Conservation")

# Mine Capacity

MineCapacity = model2.addConstrs((extract[m]
                                  <= maxproduction[m]*working[m]  for m in mine), "Capacity")

TotalProfit = model2.addConstr(gp.quicksum(price[m] * extract[m] for m in mine)
                                - gp.quicksum(royalties[m]*working[m] for m in mine)>=50000000) 

In [17]:
model2.setObjective((gp.quicksum(price[m] * extract[m] for m in mine) - gp.quicksum(royalties[m]*working[m] for m in mine)), 
                   GRB.MAXIMIZE)

In [18]:
model2.write('Q2.lp')
model2.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 17 columns and 25 nonzeros
Model fingerprint: 0x4bd9871f
Variable types: 9 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+06]
  Objective range  [8e+00, 5e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 5e+07]
Presolve removed 7 rows and 17 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 5.25e+07 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.250000000000e+07, best bound 5.250000000000e+07, gap 0.0000%


In [19]:
status = model2.Status
if status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE  or status == GRB.Status.UNBOUNDED:
    print('The model cannot be solved because it is infeasible or unbounded')
    sys.exit(0)
# If the optimization status of the model is not optimal for some other reason, we report that 
# situation.
if status != GRB.Status.OPTIMAL:
    print('Optimization was stopped with status ' + str(status))
    sys.exit(0)

In [20]:
for x in model2.getVars():
    print(x.varName, x.x)

Blend[1] 7500000.0
Blend[2] 0.0
Blend[3] 0.0
Blend[4] 0.0
Blend[5] 0.0
Extract[1] 2000000.0
Extract[2] 2500000.0
Extract[3] 0.0
Extract[4] 3000000.0
Working[1] 1.0
Working[2] 1.0
Working[3] 0.0
Working[4] 1.0
Available[1] 0.0
Available[2] 0.0
Available[3] 0.0
Available[4] 0.0


In [21]:
profit1 = 10 * 2000000.0 + 9 * 2500000.0+ 8*3000000.0

In [22]:
production1 = 7500000

In [23]:
print(f'They should operate mines 1,2,4')
print(f'The profit is {profit1}')
print(f'The production is {production1}')

They should operate mines 1,2,4
The profit is 66500000.0
The production is 7500000


### 3. Based on Q2, if mining company decides to reduce labor costs and storage costs by keeping production volumes as small as possible while maximizing total revenue. Both objectives are equally important. What should they do?

### Objective function

\begin{equation}
\text{Maximize} \quad Profit = \sum_{m \in \text{Mines}}{(\text{price}_m*\text{extract}_m-\text{royalties}_m*\text{working}_{m})}
\tag{1}
\end{equation}

\begin{equation}
\text{Minimize} \quad Production = \sum_{m \in \text{Mines}}{\text{extract}_m}
\tag{2}
\end{equation}

### Constraints

- **Operating Mines**: The total number of operating mines in year $t$ cannot exceed the limit.

\begin{equation}
\sum_{m \in \text{Mines}}{\text{working}_{m}} \leq \text{max_mines}
\tag{1}
\end{equation}

- **Mine Capacity**: Total tons of ore extracted from mine $m$ in year $t$ cannot exceed the yearly capacity of that mine.

\begin{equation}
\sum_{m \in \text{Mines}}{\text{extract}_{m}} \leq \text{capacity}_m*\text{working}_{m}
\tag{2}
\end{equation}

- **Total profit**: Total Profit cannot be less than 64M dollars

\begin{equation}
\sum_{m \in \text{Mines}}{(\text{price}_m*\text{extract}_m-\text{royalties}_m*\text{working}_{m})}>=64,000,000
\tag{3}
\end{equation}



In [24]:
total_production = gp.quicksum(extract[m] for m in mine)

In [25]:
model2.setObjective(total_production, 
                   GRB.MINIMIZE)

In [26]:
model2.write('Q2.lp')
model2.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 17 columns and 25 nonzeros
Model fingerprint: 0xca8006c2
Variable types: 9 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 5e+07]

MIP start from previous solve produced solution with objective 7.1875e+06 (0.01s)
Loaded MIP start from previous solve with objective 7.1875e+06

Presolve removed 7 rows and 17 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 7.1875e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.187500000000e+06, best bound 7.187500000000e+06, gap 0.0000%


In [27]:
status = model2.Status
if status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE  or status == GRB.Status.UNBOUNDED:
    print('The model cannot be solved because it is infeasible or unbounded')
    sys.exit(0)
# If the optimization status of the model is not optimal for some other reason, we report that 
# situation.
if status != GRB.Status.OPTIMAL:
    print('Optimization was stopped with status ' + str(status))
    sys.exit(0)

In [28]:
for x in model2.getVars():
    print(x.varName, x.x)

Blend[1] 7187500.0
Blend[2] 0.0
Blend[3] 0.0
Blend[4] 0.0
Blend[5] 0.0
Extract[1] 2000000.0
Extract[2] 2500000.0
Extract[3] 0.0
Extract[4] 2687500.0
Working[1] 1.0
Working[2] 1.0
Working[3] 0.0
Working[4] 1.0
Available[1] 0.0
Available[2] 0.0
Available[3] 0.0
Available[4] 0.0


In [29]:
profit2 = 10 * 2000000.0 + 9 * 2500000.0+ 8*2687500.0

In [30]:
production2 = 7187500.0

In [31]:
print(f'They should operate mines 1,2,4')
print(f'The profit is {profit2}')
print(f'The production is {production2}')

They should operate mines 1,2,4
The profit is 64000000.0
The production is 7187500.0


### MINIMAX Objective

In [32]:
mo = gp.Model("multiobj")

In [33]:
working2 = mo.addVars(mine, vtype=GRB.BINARY, name="Working")

In [34]:
blend = mo.addVars(year, name="Blend")
extract = mo.addVars(mine, name="Extract")
working = mo.addVars(mine, vtype=GRB.BINARY, name="Working")
available = mo.addVars(mine, vtype=GRB.BINARY, name="Available")

In [35]:
#1. Operating Mines

OperatingMines  = mo.addConstr((gp.quicksum(working2[m] for m in mine ) == max_mines), "Operating_mines")

#3. Mass Conservation

MassConservation = mo.addConstr((gp.quicksum(extract[m]for m in mine ) == blend[1]), "Mass Conservation")

#4. Mine Capacity

MineCapacity = mo.addConstrs((extract[m]
                                  <= maxproduction[m]*working2[m]  for m in mine), "Capacity")

TotalProfit = mo.addConstr(gp.quicksum(price[m] * extract[m] for m in mine)
                                - gp.quicksum(royalties[m]*working2[m] for m in mine)>=50000000) 

In [36]:
w1 = 1
w2 = 1

In [37]:
Q = mo.addVar(name = "Q", lb = -GRB.INFINITY)

In [38]:
sale = gp.quicksum(price[m] * extract[m] for m in mine)
cost = gp.quicksum(royalties[m]*working2[m] for m in mine)

In [39]:
goal1 = mo.addConstr((profit1 - w1 * (sale - cost))/profit1<= Q)


In [40]:
goal2 = mo.addConstr(w2 * ((gp.quicksum(extract[m] for m in mine)) - production2)/production2<= Q)


In [41]:
mo.setObjective(Q)
mo.write('mining-multiobj.lp')


In [42]:
# Optimize
mo.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 9 rows, 22 columns and 39 nonzeros
Model fingerprint: 0xbb4af0e8
Variable types: 10 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e-07, 5e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+07]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 9 rows and 22 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 0.210526 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.105263157895e-01, best bound 2.105263157895e-01, gap 0.0000%


In [43]:
for x in mo.getVars():
    print(x.varName, x.x)

Working[1] 1.0
Working[2] 1.0
Working[3] 0.0
Working[4] 1.0
Blend[1] 7500000.0
Blend[2] 0.0
Blend[3] 0.0
Blend[4] 0.0
Blend[5] 0.0
Extract[1] 2000000.0
Extract[2] 2500000.0
Extract[3] 0.0
Extract[4] 3000000.0
Working[1] 0.0
Working[2] 0.0
Working[3] 0.0
Working[4] 0.0
Available[1] 0.0
Available[2] 0.0
Available[3] 0.0
Available[4] 0.0
Q 0.210526315789473


In [44]:
print(f'They should operate mines 1,2,4')
print(f'The profit is {profit1}')

print(f'The answer is same as Q2')

They should operate mines 1,2,4
The profit is 66500000.0
The answer is same as Q2


 ### 4. How to maximize total revenue for five years without considering money discounted every year?

### Objective Function

- **Profit**: Maximize the total profit (in USD) of the planning horizon.

\begin{equation}
\text{Maximize} \quad Z = \sum_{t \in \text{Years}}\sum_{m \in \text{Mines}}({\text{price}*\text{blend}_t-\text{royalties}_m*\text{available}_{t,m}})
\tag{0}
\end{equation}

### Constraints

- **Operating Mines**: The total number of operating mines in year $t$ cannot exceed the limit.

\begin{equation}
\sum_{m \in \text{Mines}}{\text{working}_{t,m}} \leq \text{max_mines} \quad \forall t \in \text{Years}
\tag{1}
\end{equation}

- **Quality**: The final quality of the ore blended in year $t$ must meet the target.

\begin{equation}
\sum_{m \in \text{Mines}}{\text{quality}_m*\text{extract}_{t,m}} = \text{target}_t*\text{blended}_t \quad \forall t \in \text{Years}
\tag{2}
\end{equation}

- **Mass Conservation**: Total tons of ore extracted in year $t$ should be equal to the Tons of the ore blended in that year.

\begin{equation}
\sum_{m \in \text{Mines}}{\text{extract}_{t,m}} = \text{blend}_t \quad \forall t \in \text{Years}
\tag{3}
\end{equation}

- **Mine Capacity**: Total tons of ore extracted from mine $m$ in year $t$ cannot exceed the yearly capacity of that mine.

\begin{equation}
\sum_{m \in \text{Mines}}{\text{extract}_{t,m}} \leq \text{capacity}_m*\text{working}_{t,m} \quad \forall t \in \text{Years}
\tag{4}
\end{equation}

- **Open to Operate**: Mine $m$ can be operated in year $t$ only if it is open in that year. 

\begin{equation}
\text{working}_{t,m} \leq \text{available}_{t,m} \quad \forall (t,m) \in \text{Years} \times \text{Mines}
\tag{5}
\end{equation}

- **Shut Down**: If mine $m$ is closed in year $t$, it cannot be opened again in the future.

\begin{equation}
\text{available}_{t+1,m} \leq \text{available}_{t,m} \quad \forall (t < 5,m) \in \text{Years} \times \text{Mines}
\tag{6}
\end{equation}

In [45]:
# Parameters

years = [1, 2, 3, 4, 5]
mines = [1, 2, 3, 4]

royalties = {1: 5e6, 2: 4e6, 3: 4e6, 4: 5e6}
capacity = {1: 2e6, 2: 2.5e6, 3: 1.3e6, 4: 3e6}
quality  = {1: 1.0, 2: 0.7, 3: 1.5, 4: 0.5}
target = {1: 0.9, 2: 0.8, 3: 1.2, 4: 0.6, 5: 1.0}

max_mines = 3
price = 10

In [46]:
model4 = gp.Model('Mining')

blend = model4.addVars(years, name="Blend")
extract = model4.addVars(years, mines, name="Extract")
working = model4.addVars(years, mines, vtype=GRB.BINARY, name="Working")
available = model4.addVars(years, mines, vtype=GRB.BINARY, name="Available")

In [47]:
#1. Operating Mines

OperatingMines = model4.addConstrs((working.sum(year, '*') <= max_mines for year in years), "Operating_mines")

In [48]:
#2. Mass Conservation

MassConservation = model4.addConstrs((extract.sum(year, '*') == blend[year] \
                                      for year in years), "Mass Conservation")

In [49]:
#3. Quality

Quality = model4.addConstrs((gp.quicksum(quality[mine]*extract[year, mine] for mine in mines)
                   == target[year]*blend[year] for year in years), "Quality")

In [50]:
#4. Mine Capacity

MineCapacity = model4.addConstrs((extract[year, mine] <= capacity[mine]*working[year, mine] \
                                  for year, mine in extract), "Capacity")

In [51]:
#5. Open to operate
OpenToOperate = model4.addConstrs((working[year, mine] <= available[year, mine] \
                                   for year, mine in available), "Open to Operate")

In [52]:
#6. Shutdown Mine
ShutdownMine = model4.addConstrs((available[year+1, mine] <= available[year, mine]
                   for year, mine in available if year < years[-1]), "Shut down")

In [53]:
#0. Objective function
obj = gp.quicksum(price*blend[year] for year in years) \
- gp.quicksum(royalties[mine] * available[year, mine] for year, mine in available)
model4.setObjective(obj, GRB.MAXIMIZE)

In [54]:
model4.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 71 rows, 65 columns and 182 nonzeros
Model fingerprint: 0xd2a96bb3
Variable types: 25 continuous, 40 integer (40 binary)
Coefficient statistics:
  Matrix range     [5e-01, 3e+06]
  Objective range  [1e+01, 5e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 13 rows and 13 columns
Presolve time: 0.00s
Presolved: 58 rows, 52 columns, 135 nonzeros
Variable types: 16 continuous, 36 integer (36 binary)

Root relaxation: objective 1.875088e+08, 40 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.8751e+08    0    4   -0.00000 1.8751e+08      -     -    0s
H    0     0                    1.459167e+08

In [55]:
for x in model4.getVars():
    print(x.varName, x.x)

Blend[1] 5749999.999999999
Blend[2] 5999999.999999999
Blend[3] 3250000.0
Blend[4] 5625000.0
Blend[5] 5466666.666666666
Extract[1,1] 2000000.0
Extract[1,2] 0.0
Extract[1,3] 1300000.0
Extract[1,4] 2450000.000000001
Extract[2,1] 0.0
Extract[2,2] 2500000.0
Extract[2,3] 1300000.0
Extract[2,4] 2199999.999999999
Extract[3,1] 1950000.0000000002
Extract[3,2] 0.0
Extract[3,3] 1300000.0
Extract[3,4] 0.0
Extract[4,1] 124999.99999999997
Extract[4,2] 2500000.0
Extract[4,3] 0.0
Extract[4,4] 3000000.0
Extract[5,1] 2000000.0
Extract[5,2] 2166666.6666666665
Extract[5,3] 1300000.0
Extract[5,4] 0.0
Working[1,1] 1.0
Working[1,2] 0.0
Working[1,3] 1.0
Working[1,4] 1.0
Working[2,1] 0.0
Working[2,2] 1.0
Working[2,3] 1.0
Working[2,4] 1.0
Working[3,1] 1.0
Working[3,2] 0.0
Working[3,3] 1.0
Working[3,4] -0.0
Working[4,1] 1.0
Working[4,2] 1.0
Working[4,3] -0.0
Working[4,4] 1.0
Working[5,1] 1.0
Working[5,2] 1.0
Working[5,3] 1.0
Working[5,4] 0.0
Available[1,1] 1.0
Available[1,2] 1.0
Available[1,3] 1.0
Available[1,4] 

In [56]:
print('Obj: ' + str(model4.objVal))

Obj: 175916666.66666666


### 5. How to maximize total revenue for five years considering money discounted every year?

### Objective Function

- **Profit**: Maximize the total profit (in USD) of the planning horizon.

\begin{equation}
\text{Maximize} \quad Z = \sum_{t \in \text{Years}}\sum_{m \in \text{Mines}}{\text{time_discount}_t*(\text{price}*\text{blend}_t-\text{royalties}_m*\text{avilable}_{t,m})}
\tag{0}
\end{equation}

### Constraints

- **Operating Mines**: The total number of operating mines in year $t$ cannot exceed the limit.

\begin{equation}
\sum_{m \in \text{Mines}}{\text{working}_{t,m}} \leq \text{max_mines} \quad \forall t \in \text{Years}
\tag{1}
\end{equation}

- **Quality**: The final quality of the ore blended in year $t$ must meet the target.

\begin{equation}
\sum_{m \in \text{Mines}}{\text{quality}_m*\text{extract}_{t,m}} = \text{target}_t*\text{blended}_t \quad \forall t \in \text{Years}
\tag{2}
\end{equation}

- **Mass Conservation**: Total tons of ore extracted in year $t$ should be equal to the Tons of the ore blended in that year.

\begin{equation}
\sum_{m \in \text{Mines}}{\text{extract}_{t,m}} = \text{blend}_t \quad \forall t \in \text{Years}
\tag{3}
\end{equation}

- **Mine Capacity**: Total tons of ore extracted from mine $m$ in year $t$ cannot exceed the yearly capacity of that mine.

\begin{equation}
\sum_{m \in \text{Mines}}{\text{extract}_{t,m}} \leq \text{capacity}_m*\text{working}_{t,m} \quad \forall t \in \text{Years}
\tag{4}
\end{equation}

- **Open to Operate**: Mine $m$ can be operated in year $t$ only if it is open in that year. 

\begin{equation}
\text{working}_{t,m} \leq \text{available}_{t,m} \quad \forall (t,m) \in \text{Years} \times \text{Mines}
\tag{5}
\end{equation}

- **Shut Down**: If mine $m$ is closed in year $t$, it cannot be opened again in the future.

\begin{equation}
\text{available}_{t+1,m} \leq \text{available}_{t,m} \quad \forall (t < 5,m) \in \text{Years} \times \text{Mines}
\tag{6}
\end{equation}

In [57]:
# Parameters

years = [1, 2, 3, 4, 5]
mines = [1, 2, 3, 4]

royalties = {1: 5e6, 2: 4e6, 3: 4e6, 4: 5e6}
capacity = {1: 2e6, 2: 2.5e6, 3: 1.3e6, 4: 3e6}
quality  = {1: 1.0, 2: 0.7, 3: 1.5, 4: 0.5}
target = {1: 0.9, 2: 0.8, 3: 1.2, 4: 0.6, 5: 1.0}
time_discount = {year: (1/(1+1/10.0)) ** (year-1) for year in years}

max_mines = 3
price = 10

In [58]:
model5 = gp.Model('Mining')

blend = model5.addVars(years, name="Blend")
extract = model5.addVars(years, mines, name="Extract")
working = model5.addVars(years, mines, vtype=GRB.BINARY, name="Working")
available = model5.addVars(years, mines, vtype=GRB.BINARY, name="Available")

In [59]:
#1. Operating Mines

OperatingMines = model5.addConstrs((working.sum(year, '*') <= max_mines for year in years), "Operating_mines")

In [60]:
#2. Quality

Quality = model5.addConstrs((gp.quicksum(quality[mine]*extract[year, mine] for mine in mines)
                   == target[year]*blend[year] for year in years), "Quality")

In [61]:
#3. Mass Conservation

MassConservation = model5.addConstrs((extract.sum(year, '*') == blend[year] for year in years), "Mass Conservation")

In [62]:
#4. Mine Capacity

MineCapacity = model5.addConstrs((extract[year, mine] <= capacity[mine]*working[year, mine] for year, mine in extract), "Capacity")

In [63]:
# Open to operate
OpenToOperate = model5.addConstrs((working[year, mine] <= available[year, mine] for year, mine in available), "Open to Operate")

In [64]:
# Shutdown Mine
ShutdownMine = model5.addConstrs((available[year+1, mine] <= available[year, mine]
                   for year, mine in available if year < years[-1]), "Shut down")

In [65]:
#0. Objective function
obj = gp.quicksum(price*time_discount[year]*blend[year] for year in years) \
- gp.quicksum(royalties[mine] * time_discount[year] * available[year, mine] for year, mine in available)

model5.setObjective(obj, GRB.MAXIMIZE)

In [66]:
model5.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 71 rows, 65 columns and 182 nonzeros
Model fingerprint: 0xbc1c2f7b
Variable types: 25 continuous, 40 integer (40 binary)
Coefficient statistics:
  Matrix range     [5e-01, 3e+06]
  Objective range  [7e+00, 5e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 13 rows and 13 columns
Presolve time: 0.00s
Presolved: 58 rows, 52 columns, 135 nonzeros
Variable types: 16 continuous, 36 integer (36 binary)

Root relaxation: objective 1.577309e+08, 40 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.5773e+08    0    4   -0.00000 1.5773e+08      -     -    0s
H    0     0                    1.189074e+08

In [67]:
rows = years.copy()
columns = mines.copy()
extraction = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year, mine in extract.keys():
    if (abs(extract[year, mine].x) > 1e-6):
        extraction.loc[year, mine] = np.round(extract[year, mine].x / 1e6, 2)
extraction

,1,2,3,4
1,2.00,0.00,1.3,2.45
2,0.00,2.50,1.3,2.20
3,1.95,0.00,1.3,0.00
4,0.12,2.50,0.0,3.00
5,2.00,2.17,1.3,0.00
